<a href="https://colab.research.google.com/github/Nightcrawler9x/Beyond_visible_spectrum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge

##Generalidades

In [1]:
#@title Imports
import gdown
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [22]:
#@title Funciones
def load_from_drive(file_id, name):
  gdown.download(id=file_id, output=name, quiet=False)
  return name

def zero_pad(x, target_shape):
    padded = np.zeros(target_shape, dtype=x.dtype)
    slices = tuple(slice(0, min(dim, target)) for dim, target in zip(x.shape, target_shape))
    padded[slices] = x[slices]
    return padded

In [3]:
#@title Carga de los datos
%%capture
load_from_drive('1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn', 'data.zip')
!unzip "/content/data.zip" -d "/content/data"

##Exploracion de los datos

In [4]:
df_train = pd.read_csv('/content/data/train.csv')
df_test = pd.read_csv('/content/data/test.csv')

In [5]:
df_train.head()

,id,label
0,sample697.npy,7
1,sample54.npy,81
2,sample2270.npy,4
3,sample1401.npy,99
4,sample1901.npy,43


In [6]:
df_test.head()

,id
0,sample1957.npy
1,sample1369.npy
2,sample782.npy
3,sample2255.npy
4,sample2160.npy


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2177 non-null   object
 1   label   2177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.1+ KB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      545 non-null    object
dtypes: object(1)
memory usage: 4.4+ KB


In [16]:
print(len(df_train['label'].unique()))

101


In [37]:
dir_csv = '/content/data/train.csv'
dir_npys = '/content/data/ot/ot'

chunk_size = 32
EXPECTED_SHAPE = (128,128,125)

df = pd.read_csv(dir_csv)

npys = []
labels = []

for idx, fila in df.iterrows():
  npy_path = os.path.join(dir_npys, fila['id'])

  try:
    npy = np.load(npy_path)

    if npy.shape != EXPECTED_SHAPE:
      try:
          npy = zero_pad(npy, EXPECTED_SHAPE)
      except Exception as e:
          print(f"Error haciendo padding: {npy.shape} en {e}")
          continue

    npys.append(npy)
    labels.append(fila['label'])

    if len(npys) == chunk_size:
        X_chunk = np.array(npys)
        y_chunk = np.array(labels)

        print(f"Chunk listo: {X_chunk.shape}, etiquetas: {y_chunk.shape}")
        break
  except Exception as e:
      print(f"Error cargando {npy_path}: {e}")

Chunk listo: (32, 128, 128, 125), etiquetas: (32,)


In [38]:
X_chunk.shape

(32, 128, 128, 125)

In [39]:
y_chunk.shape

(32,)